In [2]:
pwd

'/content'

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/FIT3162')

In [5]:
!pip install pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.1 MB/s eta 0:00:00


In [6]:
import unittest

import numpy as np
import unittest
import numpy as np
import LSTM_train_eva


In [6]:
import torch
import unittest

class TestModelTraining(unittest.TestCase):

  def setUp(self):
      # Initialize a Data_util object with mock data and parameters
      self.data = LSTM_train_eva.Data_util('data/electricity.txt', 0.6, 0.2, False, 12, 24 * 7, 2)
      self.X_train, self.Y_train = self.data.train[0], self.data.train[1]# Get training data batches

      # Initialize a simple LSTM model
      self.model = LSTM_train_eva.LSTM(input_size=len(self.X_train[0][0]), hidden_size=128)

      # Initialize a loss function and optimizer
      self.criterion = torch.nn.MSELoss()
      self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

      # Move model to GPU if available
      self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.model.to(self.device)
      self.X_train = self.X_train.to(self.device)
      self.Y_train = self.Y_train.to(self.device)

  def test_model_weights_update(self):
    def train(model, data, X, Y, criterion, optimizer, batch_size):
      model.train()  # Set the model to training mode
      total_loss = 0
      n_samples = 0
      count = 0

      for X_batch, Y_batch in data.get_batches(X, Y, batch_size, True):
        #run once only
        if count < 3:
          optimizer.zero_grad()  # Clear gradients
          hidden_state, output = model(128,X_batch)  # Forward pass
          loss = criterion(output, Y_batch)  # Compute loss
          loss.backward()  # Backward pass
          optimizer.step()  # Update weights

          total_loss += loss.item()
          n_samples += X_batch.size(0)  # Increment sample count
          count += 1
      return total_loss / n_samples

    # Before training, save the initial weights
    initial_weights = [param.data.clone() for param in self.model.parameters()]

    # Perform one training step
    train(self.model, self.data, self.X_train, self.Y_train, self.criterion, self.optimizer, batch_size=128)

    # After training, check if the weights have changed
    updated_weights = [param.data.clone() for param in self.model.parameters()]

    # Check if all corresponding weights in the lists are equal
    weights_equal = all(torch.equal(iw, uw) for iw, uw in zip(initial_weights, updated_weights))

    self.assertFalse(weights_equal, "Weights should be updated during training")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
.
----------------------------------------------------------------------
Ran 1 test in 7.956s

OK


In [13]:
import torch
import unittest
from LSTM_train_eva import LSTM, LSTMCell, InputGate, ForgetGate, OutputGate # Ensure this import matches the training script

class TestModelLoadingAndPrediction(unittest.TestCase):
    def setUp(self):
        # Path to the saved model parameters
        self.model_path = 'model/model.pt'

        # Replace these with the actual input and hidden sizes of your model
        self.input_size = 10
        self.hidden_size = 128

        # Batch size for the dummy input; typically 1 for a single sequence
        self.batch_size = 1

        # Sequence length is assumed to be 1 for simplicity in this test
        self.sequence_length = 128

        # Create a dummy input tensor with the appropriate size
        # (batch_size, sequence_length, input_size)
        self.dummy_input = torch.randn(self.batch_size, self.sequence_length, self.input_size)

        # Load the model
        # Ensure the LSTM class here is the same as the one used during training
        # self.model = LSTM_train_eva.LSTM(input_size=self.input_size, hidden_size=self.hidden_size)
        self.model = torch.load(self.model_path)
        self.model.eval()  # Set the model to evaluation mode

    def test_model_prediction(self):
        """
        Test if the loaded model can make predictions on dummy input.
        """
        with torch.no_grad():
            # Perform a forward pass through the model to get predictions
            # If your model returns only the final output, you might not need the _ (underscore)
            _, predictions = self.model(128,self.dummy_input)

            # Check if the model produced any output
            self.assertIsNotNone(predictions, "The model should produce predictions")

            # Check if the predictions have the correct shape
            # The expected shape might vary depending on your model's output
            # Here we expect (batch_size, sequence_length, hidden_size)
            expected_shape = (self.dummy_input.size(0), self.dummy_input.size(1), self.hidden_size)
            self.assertEqual(predictions.shape, expected_shape, "Predictions should have the expected shape")

if __name__ == '__main__':
    # The 'first-arg-is-ignored' is a workaround for unittest when running in some IDEs
    # that may pass the script name as an argument.
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
E
ERROR: test_model_prediction (__main__.TestModelLoadingAndPrediction)
Test if the loaded model can make predictions on dummy input.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-13-4e2b55184f1c>", line 37, in test_model_prediction
    _, predictions = self.model(128,self.dummy_input)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_

In [ ]:

# class TestData(unittest.TestCase):
#     def setUp(self):
#         self.data1 = LSTM_train_eva.Data_util('data/electricity.txt', 0.6, 0.2, False, 12, 24 * 7, 2)
#         self.data2 = LSTM_train_eva.Data_util('data/solar_AL.txt', 0.7, 0.2, False, 12, 24 * 7, 0)

#     def test_data_object_creation(self):
#         print("Test data object creation")
#         self.assertIsInstance(self.data1, LSTM_train_eva.Data_util)
#         self.assertIsInstance(self.data2, LSTM_train_eva.Data_util)

#     def test_arm(self):
#         print("Test arm")
#         with open('data/electricity.txt') as file1, open('data/solar_AL.txt') as file2:
#             no_rows1 = len(np.loadtxt(file1, delimiter=',')[0])
#             no_rows2 = len(np.loadtxt(file2, delimiter=',')[0])

#         self.assertLess(len(self.data1.rawdat[0]), no_rows1)
#         self.assertEqual(len(self.data2.rawdat[0]), no_rows2)

#     def test_split(self):
#         print("Test split")
#         with open('data/electricity.txt') as file1, open('data/solar_AL.txt') as file2:
#             data1 = np.loadtxt(file1, delimiter=',')
#             data2 = np.loadtxt(file2, delimiter=',')

#         # Assuming Data_util class splits the data correctly, adjust the calculations if needed
#         no_train_cols1 = int(len(data1) * 0.6) + 1 - 24 * 7 - 12
#         no_val_cols1 = int(len(data1) * 0.2) + 1
#         no_test_cols1 = int(len(data1) * 0.2) + 1

#         no_train_cols2 = int(len(data2) * 0.7) + 1 - 24 * 7 - 12
#         no_val_cols2 = int(len(data2) * 0.2) - 1
#         no_test_cols2 = int(len(data2) * 0.1) + 1

#         self.assertEqual(len(self.data1.train[0]), no_train_cols1)
#         self.assertEqual(len(self.data1.valid[0]), no_val_cols1)
#         self.assertEqual(len(self.data1.test[0]), no_test_cols1)

#         self.assertEqual(len(self.data2.train[0]), no_train_cols2)
#         self.assertEqual(len(self.data2.valid[0]), no_val_cols2)
#         self.assertEqual(len(self.data2.test[0]), no_test_cols2)

# if __name__ == '__main__':
#     unittest.main(argv=['first-arg-is-ignored'], exit=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
EEE
ERROR: test_arm (__main__.TestData)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-6-6614ca2330cd>", line 8, in setUp
    self.data2 = LSTM_train_eva.Data_util('data/solar_AL.txt', 0.7, 0.2, False, 12, 24 * 7, 0)
  File "/content/drive/MyDrive/FIT3162/LSTM_train_eva.py", line 45, in __init__
    self._arm(5, 0.7, 0.7, 50)
  File "/content/drive/MyDrive/FIT3162/LSTM_train_eva.py", line 80, in _arm
    rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_threshold)
  File "/usr/local/lib/python3.10/dist-packages/mlxtend/freq